In [1]:
import sys
python_path = ['/home/xu.chao/cantera/build/python', '/home/xu.chao/RMG-Py', '/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages', '/home/xu.chao/cantera', '/home/xu.chao/anaconda3/envs/rmg_env/lib/python37.zip', '/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7', '/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/lib-dynload', '/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg', '/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [2]:
import cantera as ct
print(ct.__file__)

import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import rmgpy
import rmgpy.data.base
import rmgpy.molecule
import rmgpy.quantity
import scipy

/home/xu.chao/cantera/build/python/cantera/__init__.py


In [3]:
# make input list mannually
inputs = [('H', 0), ('C', 4), ('C', 2), ('C', 4), ('O', 0), ('C', 2), ('C', 3), ('C', 1), ('C', 0), ('H', 1), ('O', 1),
           ('O', 2), ('C', 2), ('C', 1), ('O', 1), ('C', 4), ('C', 1), ('C', 4), ('C', 4)]

In [4]:
benergy_c = np.arange(-5.5, -7.75, -0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)

In [5]:
def find_species_phase_index(gas, surf, species_name):
    """
    Return the phase object (gas or surface) and the index
    of the named species.
    """
    try:
        i = gas.species_index(species_name)
        return gas, i
    except ValueError:
        i = surf.species_index(species_name)
        return surf, i
    
def change_species_enthalpy(gas, surf, species_name, dH, temp):
    """
    Find the species by name and change it's enthlapy by dH (in J/kmol)
    """
    phase, index = find_species_phase_index(gas, surf, species_name)

    species = phase.species(index)
    print(f"Initial H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    dx = dH / ct.gas_constant  # 'dx' is in fact (delta H / R). Note that R in cantera is 8314.462 J/kmol
    assert isinstance(species.thermo, ct.NasaPoly2)
    # print(species.thermo.coeffs)
    perturbed_coeffs = species.thermo.coeffs.copy()
    perturbed_coeffs[6] += dx
    perturbed_coeffs[13] += dx
    
    species.thermo = ct.NasaPoly2(species.thermo.min_temp, species.thermo.max_temp, 
                            species.thermo.reference_pressure, perturbed_coeffs)
#     print(species.thermo.coeffs)
    phase.modify_species(index, species)
    print(f"Modified H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    
def correct_binding_energy(surf, gas, target_surf, spec_index, center_atom, x, temperature):
    """
    Here applies linear scaling relationship to calculate binding energy (J/kmol) from one metal
    to another
    Args:
        target_surf: binding energy dictionary of interest eg: Pt_111
        spec_index: index of the species to change
        center_atom: center atom of the species to change
        x: bonds of center atoms of the species to change
    """
    pt_111 = {'H': (-2.75368,'eV/molecule'),
              'C': (-7.02516,'eV/molecule'),
              'N': (-4.63225,'eV/molecule'),
              'O': (-3.81153,'eV/molecule'),
             }
    x_max = {'H': 1,
             'C': 4,
             'N': 3,
             'O': 2,
            }
    deltaE = ((x_max[center_atom]-x) / x_max[center_atom]) * (target_surf[center_atom][0]-pt_111[center_atom][0])*9.6487e7
    print(f"\nApplying LSR correction to {surf.species(spec_index).name}:")
    print(f"Changing H by {deltaE/1e6:+.1f} kJ/mol   ({deltaE/96487000.:+.2f} eV)")
    change_species_enthalpy(gas, surf, surf.species(spec_index).name, deltaE, temperature)

In [6]:
index=0.0
for c_binding_energy in benergy_c:
    for o_binding_energy in benergy_o:
        index += 1.0
        print(f'Converting C={c_binding_energy}, O={o_binding_energy}')
        path_to_cti = 'base_bm.yaml'
        gas1 = ct.Solution(path_to_cti, 'gas')
        surf1 = ct.Interface(path_to_cti, 'surface1', [gas1])
        be_dic = {'C': (c_binding_energy, 'eV/molecule'),
                  'O': (o_binding_energy, 'eV/molecule'),
                  'H': (-2.75368,'eV/molecule')
                 }
        for i in range(1, surf1.n_species, 1):
            correct_binding_energy(surf1, gas1, be_dic, i, inputs[i-1][0], inputs[i-1][1], 700)
        data_root = 'binding_energies/{}_c{:.2f}o{:.2f}'.format(index, c_binding_energy, o_binding_energy)
        if not os.path.exists(data_root):
            os.mkdir(data_root)  
#         gas.write_yaml(os.path.join(data_root, f'dm_cpox_c{c_binding_energy}_o{o_binding_energy}.yaml'))
        surf1.write_yaml(os.path.join(data_root, 'cantera.yaml'))

Converting C=-5.5, O=-3.25

Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -191.5 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +54.2 kJ/mol   (+0.56 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -77.3 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 90.7 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +36.8 kJ/mol   (+0.38 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 11.3 kJ/mol

Apply

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in cp/R detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  10.615319744584205
	Value computed using high-temperature polynomial: 10.773680323194302

  
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -16.685067677440152
	Value computed using high-temperature polynomial: -16.696975757668092

  
/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: NasaPoly2::validate: 
For species CH4OX(43), discontinuity in cp/R detected at Tmid = 850.41
	Value computed using low-temperature polynomial:  11.305535344771027
	Value computed using high-temperature polynomial:


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -191.5 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +30.1 kJ/mol   (+0.31 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -101.4 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 90.7 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +36.8 kJ/mol   (+0.38 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 11.3 kJ/mol

Applying LSR correction to CHX(

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -16.268781899829996
	Value computed using high-temperature polynomial: -16.280689980057943

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -17.96017543798274
	Value computed using high-temperature polynomial: -17.972083518210688




Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -191.5 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by -18.2 kJ/mol   (-0.19 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -149.7 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +73.6 kJ/mol   (+0.76 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 90.7 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +36.8 kJ/mol   (+0.38 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 11.3 kJ/mol

Applying LSR correction to CHX(

...


Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by +1.2 kJ/mol   (+0.01 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -263.8 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +5.9 kJ/mol   (+0.06 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -125.5 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by +1.2 kJ/mol   (+0.01 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 18.3 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by +0.6 kJ/mol   (+0.01 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -24.8 kJ/mol

Applying LSR correction to CHX(28)

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: NasaPoly2::validate: 
For species C2H3X(244), discontinuity in h/RT detected at Tmid = 808.69
	Value computed using low-temperature polynomial:  -9.883597089909859
	Value computed using high-temperature polynomial: -9.907865543717518




Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by -10.8 kJ/mol   (-0.11 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -275.9 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +30.1 kJ/mol   (+0.31 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -101.4 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by -10.8 kJ/mol   (-0.11 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 6.3 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by -5.4 kJ/mol   (-0.06 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -30.9 kJ/mol

Applying LSR correction to CHX(2

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: NasaPoly2::validate: 
For species C2H3X(244), discontinuity in h/RT detected at Tmid = 808.69
	Value computed using low-temperature polynomial:  -12.574226082711682
	Value computed using high-temperature polynomial: -12.598494536519349




Applying LSR correction to HX(21):
Changing H by +0.0 kJ/mol   (+0.00 eV)
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol

Applying LSR correction to CO2X(22):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol

Applying LSR correction to COX(23):
Changing H by -22.9 kJ/mol   (-0.24 eV)
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -288.0 kJ/mol

Applying LSR correction to CH4X(24):
Changing H by -0.0 kJ/mol   (-0.00 eV)
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol

Applying LSR correction to OX(25):
Changing H by +5.9 kJ/mol   (+0.06 eV)
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -125.5 kJ/mol

Applying LSR correction to CH2X(26):
Changing H by -22.9 kJ/mol   (-0.24 eV)
Initial H(700) = 17.1 kJ/mol
Modified H(700) = -5.8 kJ/mol

Applying LSR correction to CH3X(27):
Changing H by -11.5 kJ/mol   (-0.12 eV)
Initial H(700) = -25.4 kJ/mol
Modified H(700) = -36.9 kJ/mol

Applying LSR correction to CHX(

In [7]:

# add elements block into yaml files
energy_c = np.arange(-5.5, -7.75, -0.25)
energy_o = np.arange(-3.25, -5.5, -0.25)
with open('base_bm.yaml', 'r') as f:
    model = yaml.load(f, Loader=yaml.FullLoader)
index = 0.0
for i in energy_c:
    for j in energy_o:
        index += 1.0
        path_to_cti = 'binding_energies/{}_c{:.2f}o{:.2f}/cantera.yaml'.format(index, i, j)
        with open(path_to_cti, 'r') as f:
            model_to_change = yaml.load(f, Loader=yaml.FullLoader)

        model_to_change['elements'] = model['elements']

        with open(path_to_cti, 'w') as f:
            yaml.dump(model_to_change, f, default_flow_style=False)
            
        gas = ct.Solution(path_to_cti, 'gas')
        surf = ct.Interface(path_to_cti, 'surface1', [gas])

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in cp/R detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  10.615319744584205
	Value computed using high-temperature polynomial: 10.773680323194302

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: NasaPoly2::validate: 
For species CH3O_X(45), discontinuity in h/RT detected at Tmid = 857.63
	Value computed using low-temperature polynomial:  -12.885994823524513
	Value computed using high-temperature polynomial: -12.897902903752454

/home/xu.chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: NasaPoly2::validate: 
For species CH4OX(43), discontinuity in cp/R detected at Tmid = 850.41
	Value computed using low-temperature polynomial:  11.305535344771027
	Value computed using high-temperature polynomial: 11

In [6]:
from datetime import datetime
benergy_c = np.arange(-5.5, -7.75, -0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)
index=0.0
for c_binding_energy in benergy_c:
    for o_binding_energy in benergy_o:
        index += 1.0
        data_root = 'binding_energies/{}_c{:.2f}o{:.2f}/sim_data/rtol_1e-08_atol_1e-08_data.csv'.format(index, c_binding_energy, o_binding_energy)
        print(data_root)
        try:
            print(datetime.fromtimestamp(os.path.getmtime(data_root)))
        except Exception as e:
            print(e)

binding_energies/1.0_c-5.50o-3.25/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:41:20.455682
binding_energies/2.0_c-5.50o-3.50/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42:18.402102
binding_energies/3.0_c-5.50o-3.75/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42:19.312310
binding_energies/4.0_c-5.50o-4.00/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42:11.900515
binding_energies/5.0_c-5.50o-4.25/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:41:44.579418
binding_energies/6.0_c-5.50o-4.50/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42:09.547507
binding_energies/7.0_c-5.50o-4.75/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42:42.822765
binding_energies/8.0_c-5.50o-5.00/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42:17.237015
binding_energies/9.0_c-5.50o-5.25/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:45:21.427489
binding_energies/10.0_c-5.75o-3.25/sim_data/rtol_1e-08_atol_1e-08_data.csv
2022-08-11 13:42

In [5]:
from datetime import datetime
benergy_c = np.arange(-5.5, -7.75, -0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)
index=0.0
for c_binding_energy in benergy_c:
    for o_binding_energy in benergy_o:
        index += 1.0
        data_root = '../base_original/binding_energies/{}_c{:.2f}o{:.2f}/sim_data/rtol_1e-08_atol_1e-16_data.csv'.format(index, c_binding_energy, o_binding_energy)
        data_root2 = '../base_original/binding_energies/{}_c{:.2f}o{:.2f}/sim_data/rtol_1e-10_atol_1e-20_data.csv'.format(index, c_binding_energy, o_binding_energy)
        print(data_root)
        try:
            print(datetime.fromtimestamp(os.path.getmtime(data_root)))
        except Exception as e:
            print(datetime.fromtimestamp(os.path.getmtime(data_root2)))

../base_original/binding_energies/1.0_c-5.50o-3.25/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:51.019564
../base_original/binding_energies/2.0_c-5.50o-3.50/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:25.499120
../base_original/binding_energies/3.0_c-5.50o-3.75/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:36.814602
../base_original/binding_energies/4.0_c-5.50o-4.00/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:43.991167
../base_original/binding_energies/5.0_c-5.50o-4.25/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:36.333584
../base_original/binding_energies/6.0_c-5.50o-4.50/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:38.839857
../base_original/binding_energies/7.0_c-5.50o-4.75/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:16.587506
../base_original/binding_energies/8.0_c-5.50o-5.00/sim_data/rtol_1e-08_atol_1e-16_data.csv
2022-08-02 17:41:13.116136
../base_original/binding_energies/9.0_c-5.50o-5.25/sim_d

In [5]:
benergy_c = np.arange(-5.5, -7.75, -0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)
index=0.0
for c_binding_energy in benergy_c:
    for o_binding_energy in benergy_o:
        index += 1.0
        data_root = 'binding_energies/{}_c{:.2f}o{:.2f}/sim_data/rtol_1e-08_atol_1e-16_data.csv'.format(index, c_binding_energy, o_binding_energy)
        data_root2 = 'binding_energies/{}_c{:.2f}o{:.2f}/sim_data/rtol_1e-10_atol_1e-20_data.csv'.format(index, c_binding_energy, o_binding_energy)
        root = 'binding_energies/{}_c{:.2f}o{:.2f}/sim_data'.format(index, c_binding_energy, o_binding_energy)
        print(os.listdir(root))

['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-05_atol_1e-10_data.csv', 'rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-20_data.csv', 'therm_sens_ref.csv']
['rtol_1e-08_atol_1e-16_data.csv', 'rtol_1e-10_atol_1e-2